In [ ]:
import os
import pandas as pd
import numpy as np
from radiomics import featureextractor, imageoperations
import nibabel as nib
import SimpleITK as sitk
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif, RFE, VarianceThreshold
from sklearn.metrics import (accuracy_score, precision_score, 
                            recall_score, f1_score, confusion_matrix, 
                            roc_curve, roc_auc_score, auc, RocCurveDisplay)
from sklearn.decomposition import PCA
from scipy.stats import levene, ttest_ind, pearsonr, skew, kurtosis
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pingouin as pg
import warnings
warnings.filterwarnings('ignore')
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号


In [ ]:
def feature_select():
	kinds = ['TUMOR','HEALTHY']
	# 这个是特征处理配置文件，具体可以参考pyradiomics官网
	para_path = 'yaml/MR_2mm.yaml'

	extractor = featureextractor.RadiomicsFeatureExtractor(para_path) 
	dir = 'data/MyData/'

	for kind in kinds:
		print("{}:开始提取特征".format(kind))
		df = pd.DataFrame()
		path =  dir + kind
		# 使用配置文件初始化特征抽取器
		for index, folder in enumerate(os.listdir(path)):
			if folder == '.DS_Store':
				continue
			origin_path = os.path.join(path, folder, 'phiMap.nii')
			file_name = folder[:-4] if folder[-3:]=='ROI' else folder
			roi_path = os.path.join(path, folder, file_name + '.nii.gz')
			features = extractor.execute(origin_path,roi_path)  #抽取特征
			#新增一列用来保存病例文件夹名字
			features = {'patient_name': file_name, **features}
			df_add = pd.DataFrame.from_dict(features.values()).T
			df_add.columns = features.keys()
			df = pd.concat([df, df_add])
		df.to_csv('results/' +'phi_{}.csv'.format(kind),index=0)
		print('提取特征完成')

feature_select()

TUMOR:开始提取特征
提取特征完成
HEALTHY:开始提取特征
提取特征完成


In [ ]:
def drawplot(image_path, roi_path, pdf, file_name):
	# 读取医学图像
	image = sitk.ReadImage(image_path)
	roi_mask = sitk.ReadImage(roi_path)

	# 转换为numpy数组
	original_array = sitk.GetArrayFromImage(image)
	roi_array = sitk.GetArrayFromImage(roi_mask)

	# 如果图像是3D的，选择需要的切片（例如中间切片）
	# if len(original_array.shape) == 3:
	# 	slice_idx = original_array.shape[0] // 3
	# 	original_slice = original_array[slice_idx]
	# 	roi_slice = roi_array[slice_idx]
	# else:
	# 	original_slice = original_array
	# 	roi_slice = roi_array

	original_slice = original_array.flatten()
	roi_slice = roi_array.flatten()

	roi_binary = (roi_slice > 0).astype(np.uint8)
	roi_pixels = original_slice[roi_binary > 0]
	statistics = {
		'像素数量': len(roi_pixels),
		'平均值': np.mean(roi_pixels),
		'中位数': np.median(roi_pixels),
		'标准差': np.std(roi_pixels),
		'最小值': np.min(roi_pixels),
		'最大值': np.max(roi_pixels),
		'范围': np.ptp(roi_pixels),
		'偏度': skew(roi_pixels),
		'峰度': kurtosis(roi_pixels),
		'第10百分位数': np.percentile(roi_pixels, 10),
		'第25百分位数': np.percentile(roi_pixels, 25),
		'第50百分位数': np.percentile(roi_pixels, 50),
		'第75百分位数': np.percentile(roi_pixels, 75),
		'第90百分位数': np.percentile(roi_pixels, 90),
		'第95百分位数': np.percentile(roi_pixels, 95),
		'方差': np.var(roi_pixels)
		}

	# 创建直方图
	fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(15, 12))

	ax1.hist(roi_pixels.flatten(), bins=10, color='gray', alpha=0.7, edgecolor='black')

	ax1.set_title(f"{file_name}ROI灰度分布直方图")
	ax1.set_xlabel('灰度值')
	ax1.set_ylabel('频数')
	ax1.grid(alpha=0.3)

	# 添加统计信息
	ax1.axvline(statistics["平均值"], color='red', linestyle='--', label=f'平均值: {statistics["平均值"]:.2f}')
	# plt.axvline(mean_val + std_val, color='orange', linestyle=':', label=f'mean±deviation')
	# plt.axvline(mean_val - std_val, color='orange', linestyle=':')
	ax1.legend()


	ax2.axis('off')
	table_data = [
		['指标', '值'],
		['像素数量', f"{statistics['像素数量']}"],
		['平均值', f"{statistics['平均值']:.2f}"],
		['标准差', f"{statistics['标准差']:.2f}"],
		['偏度', f"{statistics['偏度']:.4f}"],
		['峰度', f"{statistics['峰度']:.4f}"],
		['P10', f"{statistics['第10百分位数']:.2f}"],
		['P25', f"{statistics['第25百分位数']:.2f}"],
		['P50', f"{statistics['第50百分位数']:.2f}"],
		['P75', f"{statistics['第75百分位数']:.2f}"],
		['P90', f"{statistics['第90百分位数']:.2f}"],
		['最小值', f"{statistics['最小值']}"],
		['最大值', f"{statistics['最大值']}"]
	]
	table = ax2.table(cellText=table_data, loc='center', cellLoc='left')
	table.auto_set_font_size(False)
	table.set_fontsize(10)
	table.scale(1, 2)
	ax2.set_title('统计指标汇总')

	plt.tight_layout()
	pdf.savefig()
	# 关闭当前图形，避免内存泄漏
	plt.close()


def drawplots():
	kinds = ['TUMOR','HEALTHY']
	dir = 'data/MyData/'
	with PdfPages('results/phi_plots.pdf') as pdf:
		for kind in kinds:
			path =  dir + kind
			# 使用配置文件初始化特征抽取器
			for index, folder in enumerate(os.listdir(path)):
				if folder == '.DS_Store':
					continue
				origin_path = os.path.join(path, folder, 'phiMap.nii')
				file_name = folder[:-4] if folder[-3:]=='ROI' else folder
				roi_path = os.path.join(path, folder, file_name + '.nii.gz')
				drawplot(origin_path, roi_path, pdf, file_name)

with PdfPages('results/test1.pdf') as pdf:
	drawplot('data/MyData/TUMOR/BAO AI MEI ROI/cMap.dcm', 'data/FUYIROI/BAO AI MEI/BAO AI MEI.nii.gz', pdf, 'BAI AI MEI')
# drawplots()

[[[0.498173 0.546173 0.485173 ... 0.474173 0.419173 0.486173]
  [0.460173 0.451173 0.436173 ... 0.570173 0.494173 0.518173]
  [0.460173 0.493173 0.532173 ... 0.803173 0.731173 0.572173]
  ...
  [0.541173 0.677173 0.722173 ... 0.649173 0.670173 0.523173]
  [0.475173 0.557173 0.525173 ... 0.491173 0.595173 0.528173]
  [0.442173 0.528173 0.574173 ... 0.463173 0.498173 0.565173]]

 [[0.401173 0.392173 0.443173 ... 0.484173 0.471173 0.514173]
  [0.398173 0.440173 0.554173 ... 0.637173 0.600173 0.575173]
  [0.507173 0.469173 0.590173 ... 0.887173 0.931173 0.544173]
  ...
  [0.427173 0.540173 0.649173 ... 0.592173 0.571173 0.504173]
  [0.471173 0.561173 0.646173 ... 0.661173 0.615173 0.572173]
  [0.498173 0.542173 0.533173 ... 0.591173 0.542173 0.515173]]

 [[0.322173 0.420173 0.472173 ... 0.493173 0.486173 0.460173]
  [0.416173 0.459173 0.587173 ... 0.551173 0.550173 0.451173]
  [0.492173 0.453173 0.614173 ... 0.616173 0.532173 0.496173]
  ...
  [0.541173 0.666173 0.763173 ... 0.829173 0.656

In [ ]:
# icc-intraclass correlation coefficient-组内相关系数
# 两个医生针对同一图片画的roi，计算相关性
def icc_cal():
	data_1 = pd.read_csv('')
	data_2 = pd.read_csv('')

	features = data_1.columns.values.tolist()

	data_1.insert(0, 'doctor', np.ones(data_1.shape[0]))
	data_2.insert(0, 'doctor', np.ones(data_2.shape[0])*2)
	data = pd.concat([data_1,data_2])

	for feature in features[2:]:
		icc = pg.intraclass_corr(data = data, targets = "patient_name", raters = "doctor",ratings = feature)
		print(f"feature name: {feature}")
		print(icc)



In [ ]:
# 相关系数热图Heatmap
def corr_cal(path, type='pearson'):
	# method 参数默认计算 pearson相关系数，其它还可以计算“spearman”,"kendall"相关系数
	df = pd.read_csv(path)
	corr = df.corr(type)
	print(corr.head(5))
	plt.figure(figsize = (12,10),dpi = 80)
	sns.heatmap(corr,xticklabels = corr.columns, yticklabels = corr.columns, cmap = "RdYlGn",center = 0, annot = True)
	# clustermap
	# sns.clustermap(corr,xticklabels = corr.columns, yticklabels = corr.columns, cmap = "RdYlGn",center = 0)
	
	plt.title('Correlogram',fontsize = 22)
	plt.xticks(fontsize = 12)
	plt.yticks(fontsize = 12)
	plt.show()

In [5]:
tumor_filePath = 'results/TUMOR.csv'
healthy_filePath = 'results/HEALTHY.csv'
tumor_data = pd.read_csv(tumor_filePath)
healthy_data = pd.read_csv(healthy_filePath)
rows_1,__ = tumor_data.shape
rows_2,__ = healthy_data.shape
tumor_data.insert(0,'label',[1]*rows_1)
healthy_data.insert(0,'label',[0]*rows_2)
#因为有些特征是字符串，直接删掉
cols=[x for i,x in enumerate(tumor_data.columns) if type(tumor_data.iat[1,i]) == str]
cols.remove('patient_name')
tumor_data=tumor_data.drop(cols,axis=1)
cols=[x for i,x in enumerate(healthy_data.columns) if type(healthy_data.iat[1,i]) == str]
cols.remove('patient_name')
healthy_data=healthy_data.drop(cols,axis=1)

data = pd.concat([tumor_data,healthy_data])
data = shuffle(data)
data = data.fillna(0)
data.to_csv('results/TotalOMICS.csv',index=False)
# 划分训练集和测试集
train_data, test_data = train_test_split(data, test_size=0.2, random_state=888)
# 保存划分后的数据
train_data.to_csv("results/trainOmics.csv", index=False)
test_data.to_csv("results/testOmics.csv", index=False)

In [ ]:
from radiomics import imageoperations
imageoperations.getBinEdges()


In [ ]:
# 标准化 (影像组学中最常用)
# z = (x - u)/s
# 异常值一般对结果影响不大
trainData = pd.read_csv("results/trainOmics.csv")
X = trainData[trainData.columns[2:]]
y = trainData['label']
colNames = X.columns
X = X.astype(np.float64)
X = StandardScaler().fit_transform(X) #new knowledge
X = pd.DataFrame(X)
X.columns = colNames

In [ ]:
# 方差选择法
def variance_threshold(X):
	selector = VarianceThreshold(1e10)  # 注意修改参数达到筛选目的
	selector.fit_transform(X)
	# print('EveryVaris:'+str(selector.variances_))
	# print('selectedFeatureIndex:'+str(selector.get_support(True)))
	print('selectedFeatureNameis:'+str(X.columns[selector.get_support(True)]))
	# print('excludedFeatureNameis:'+str(X.columns[~ selector.get_support()]))  # ‘~’取反 

variance_threshold(X)

In [ ]:
# t检验,检验两组独立样本的平均数与其分布是否具有显著性差异
def t_test(data_1, data_2):
	data_1 = pd.read_csv('results/TUMOR.csv')
	data_2 = pd.read_csv('results/HEALTHY.csv')
	index = []
	for colName in data.columns[2:]:
		if levene(data_1[colName],data_2[colName])[1] > 0.05:
			if ttest_ind(data_1[colName],data_2[colName])[1] < 0.05:
				index.append(colName)
		else:
			if ttest_ind(data_1[colName],data_2[colName],equal_var = False)[1] < 0.05:
				index.append(colName)
	print(index)

In [ ]:
# LASSO特征选择
def lasso_feature_selection(X, y):
	alphas = np.logspace(-3,1,50)
	model_lassoCV = LassoCV(alphas = alphas, cv = 10, max_iter = 100000).fit(X,y)
	coef = pd.Series(model_lassoCV.coef_,index = X.columns)
	print(model_lassoCV.alpha_)
	index = coef[coef != 0].index
	X = X[index]
	# print(X.head())
	print(coef[coef != 0])

In [ ]:
# ANOVA特征选择
def anova_feature_selection(X, y, alpha=0.05, top_k=None):
	"""
    自定义ANOVA特征选择函数
	适用场景：ANOVA适用于分类问题中的连续特征选择
	数据假设：
	特征应该是连续型的
	数据应该近似正态分布
	各组方差应该相等（方差齐性）
    
    参数:
    X: 特征DataFrame
    y: 目标变量
    alpha: 显著性水平
    top_k: 选择前k个特征，如果为None则使用p值阈值
    
    返回:
    selected_features: 选中的特征列表
    results_df: 详细结果DataFrame
    """
	f_scores, p_values = f_classif(X, y)
    
    # 创建结果表格
	results_df = pd.DataFrame({
        'feature': X.columns,
        'f_score': f_scores,
        'p_value': p_values,
        'significant': p_values < alpha
    }).sort_values('f_score', ascending=False)
    
    # 选择特征
	if top_k is not None:
        # 选择前k个特征
		selected_features = results_df.head(top_k)['feature'].tolist()
	else:
        # 基于p值阈值选择
		selected_features = results_df[results_df['significant']]['feature'].tolist()
	print(f'ANOVA特征选择: {selected_features}')

In [ ]:
def RFE__feature_selection(X, y):
	# estimator = LogisticRegression(max_iter=1000, random_state=42)
	# estimator = RandomForestClassifier(random_state=42)
	estimator = RFE(
		SVC(kernel='linear', random_state=42),
		n_features_to_select=2,# 要选择的特征数量
		step=0.05,# 每次迭代移除的特征数量
		verbose=1# 显示进度
		)
	estimator.fit(X, y)
	# print("特征排名 (1表示被选中):", estimator.ranking_)
	# print("特征支持掩码:", estimator.support_)
	print("选中的特征:", X.columns[estimator.support_].tolist())

In [ ]:
# PCA(Principal component analysis)主成分分析
# 降维包括特征筛选
def pca_selection(X, y):
	X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
	model_pca = PCA(n_components = 0.99)
	model_pca.fit(X_train)
	print(model_pca.explained_variance_)
	print(model_pca.explained_variance_ratio_)

	# 将降维后的数据进行训练
	# X_train_pca = model_pca.transform(X_train)
	# X_test_pca = model_pca.transform(X_test)
	# print(X_train_pca.shape,X_test_pca.shape)
	# model_svm = svm.SVC(kernel = 'rbf',gamma = "auto",probability = True).fit(X_train_pca,y_train)
	# score_svm = model_svm.score(X_test_pca,y_test)
	# print(score_svm)

In [ ]:
# RandomFrorest
def random_forest(X, y):
	X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
	model_rf = RandomForestClassifier(n_estimators = 20).fit(X_train,y_train)
	score_rf = model_rf.score(X_test,y_test)
	print(score_rf)

In [ ]:
# 支持向量机
def svm_classifier(X, y):
	#params opt: svm 参数优化
	Cs = np.logspace(-1,3,10,base = 2)
	gammas = np.logspace(-4,1,50,base = 2)
	param_grid = dict(C = Cs, gamma = gammas)
	grid = GridSearchCV(SVC(kernel = 'rbf'),param_grid = param_grid, cv = 10).fit(X,y)
	print(grid.best_params_)
	C = grid.best_params_['C']
	gamma = grid.best_params_['gamma']

	#svm
	X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
	model_svm = SVC(kernel = 'rbf',C = C, gamma = gamma,probability = True).fit(X_train,y_train)
	score_svm = model_svm.score(X_test,y_test)
	print(score_svm)

In [ ]:
# ROC曲线及其绘制
def roc_plot():
	y_probs = model_svm.predict_proba(X)
	fpr,tpr,thresholds = roc_curve(y,y_probs[:,1],pos_label = 1)

	plt.plot(fpr,tpr,marker = 'o')
	plt.xlabel('fpr')
	plt.ylabel('tpr')
	plt.show()

	auc_score = roc_auc_score(y, y_probs[:,1])
	print(auc_score)
	#select the best threshold
	# J = tpr - fpr
	# idx = argmax(J)
	# best_threshold = thresholds[idx]